In [1]:
!pip install mediapipe
!pip install opencv-python


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [9]:
import cv2
import mediapipe as mp
import json

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen
image = cv2.imread('../yolo/videoframe_0.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Realizar la detección de pose
results = pose.process(image_rgb)

blazepose_results = []

# Dibujar las anotaciones de la pose en la imagen
if results.pose_landmarks:
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    #guardar en annotations
    keypoints = []
    for landmark in results.pose_landmarks.landmark:
        keypoints.extend([landmark.x * image.shape[1], landmark.y * image.shape[0], landmark.visibility])
    
    ann = {
        "image_id": "videoframe",
        "category_id": 1,  # Persona
        "keypoints": keypoints,
        "score": 1.0  # Puedes ajustar el score según sea necesario
    }
    blazepose_results.append(ann)

#guardar la img
cv2.imwrite('testBlazePose.jpg', image)
# Mostrar la imagen con las poses detectadas
cv2.imshow('BlazePose Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results.json', 'w') as f:
    json.dump(blazepose_results, f)


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [11]:
#usando el dataset de wholeBody
import os

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directorio de imágenes y archivo de anotaciones
image_dir = '../../val2017/val2017'  # Ajusta según la estructura de tu directorio
#annotation_file = '../../coco_wholebody_val_v1.0.json'
annotation_file = '../../person_keypoints_val2017.json'
blazepose_results = []

# Cargar el dataset COCO
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)

# Procesar las imágenes
for img_info in coco_data['images']:
    image_path = os.path.join(image_dir, img_info['file_name'])
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        keypoints = []
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x * image.shape[1], landmark.y * image.shape[0], landmark.visibility])
        
        ann = {
            "image_id": img_info['id'],
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results-wholeBody.json', 'w') as f:
    json.dump(blazepose_results, f)

#guardar una img para ver como la analiza blazepose
cv2.imwrite('testBlazePoseWholeBody.jpg', image) #seria la ultima imagen procesada del dataset


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


True

In [16]:
import cv2
import mediapipe as mp
import json
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directorio de imágenes y archivo de anotaciones
image_dir = '../../val2017/val2017'  # Ajusta según la estructura de tu directorio
annotation_file = '../yolo/datasets/coco-pose/annotations/person_keypoints_val2017.json'
blazepose_results = []

# Cargar el dataset COCO
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)
coco = COCO(annotation_file)

# Función para convertir 33 keypoints de BlazePose a 17 keypoints de COCO
def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints

# Procesar las imágenes
for img_info in coco_data['images']:
    image_path = os.path.join(image_dir, img_info['file_name'])
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Dibujar las anotaciones de la pose en la imagen
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, image.shape)
        
        ann = {
            "image_id": img_info['id'],
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results-wholeBody.json', 'w') as f:
    json.dump(blazepose_results, f)

# Guardar y mostrar una imagen procesada
if 'image' in locals():
    # Guardar la última imagen procesada con las anotaciones
    cv2.imwrite('testBlazePoseWholeBody.jpg', image)
    # Mostrar la imagen con las poses detectadas
    cv2.imshow('BlazePose WholeBody Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Evaluar las métricas de rendimiento usando pycocotools
coco_dt = coco.loadRes('blazepose_results-wholeBody.json')
coco_eval = COCOeval(coco, coco_dt, 'keypoints')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Imprimir métricas relevantes
print("mAP: ", coco_eval.stats[0])
print("AP@0.50: ", coco_eval.stats[1])
print("AP@0.75: ", coco_eval.stats[2])
print("AP (M): ", coco_eval.stats[3])
print("AP (L): ", coco_eval.stats[4])
print("AR: ", coco_eval.stats[8])

print("Anotaciones guardadas en: blazepose_results-wholeBody.json")
if 'image' in locals():
    print("Imagen procesada guardada en: testBlazePoseWholeBody.jpg")


loading annotations into memory...
Done (t=0.66s)
creating index...
index created!


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.164
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.087
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.170
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.132
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.201
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.140
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.039
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDe

In [45]:
import cv2
import mediapipe as mp
import json
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directorio de imágenes y archivo de anotaciones
image_dir = '../../COCO/val2017/val2017'  # Ajusta según la estructura de tu directorio
annotation_file = '../../person_keypoints_val2017.json'
blazepose_results = []

# Cargar el dataset COCO
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)
coco = COCO(annotation_file)

def average_landmarks_three(landmark1, landmark2, landmark3):
    return {
        'x': (landmark1.x + landmark2.x + landmark3.x) / 3,
        'y': (landmark1.y + landmark2.y + landmark3.y) / 3,
        'visibility': (landmark1.visibility + landmark2.visibility + landmark3.visibility) / 3
    }

def average_landmarks(landmark1, landmark2):
    return {
        'x': (landmark1.x + landmark2.x) / 2,
        'y': (landmark1.y + landmark2.y) / 2,
        'visibility': (landmark1.visibility + landmark2.visibility) / 2
    }

# Función para convertir 33 keypoints de BlazePose a 25 keypoints de COCO
def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 25
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[20] = landmarks[30]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[32]  # Custom point
    keypoints[23] = landmarks[9]  # Custom point
    keypoints[24] = landmarks[10]  # Custom point

    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])
    return coco_keypoints

# Procesar las imágenes
for img_info in coco_data['images']:
    image_path = os.path.join(image_dir, img_info['file_name'])
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Dibujar las anotaciones de la pose en la imagen
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, image.shape)
        
        ann = {
            "image_id": img_info['id'],
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results-wholeBody_v2.json', 'w') as f:
    json.dump(blazepose_results, f)

# Guardar y mostrar una imagen procesada
if 'image' in locals():
    # Guardar la última imagen procesada con las anotaciones
    cv2.imwrite('testBlazePoseWholeBody.jpg', image)
    # Mostrar la imagen con las poses detectadas
    cv2.imshow('BlazePose WholeBody Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Evaluar las métricas de rendimiento usando pycocotools
coco_dt = coco.loadRes('blazepose_results-wholeBody_v2.json')
coco_eval = COCOeval(coco, coco_dt, 'keypoints')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Imprimir métricas relevantes
print("mAP: ", coco_eval.stats[0])
print("AP@0.50: ", coco_eval.stats[1])
print("AP@0.75: ", coco_eval.stats[2])
print("AP (M): ", coco_eval.stats[3])
print("AP (L): ", coco_eval.stats[4])
print("AR: ", coco_eval.stats[8])

print("Anotaciones guardadas en: blazepose_results-wholeBody_v2.json")
if 'image' in locals():
    print("Imagen procesada guardada en: testBlazePoseWholeBodyv2.jpg")


loading annotations into memory...
Done (t=0.40s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*


ValueError: operands could not be broadcast together with shapes (25,) (17,) 

In [ ]:
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Define area to mark (e.g., a bounding box)
rectangles = [
    (541, 604, 587, 630),  # Coordenadas de los círculos en el piso
    (171, 525, 218, 550),
    (162, 390, 197, 408),
    (923, 373, 953, 389),
    (905, 504, 946, 526),
    (768, 299, 793, 311),
    (338, 305, 367, 319),
    (553, 381, 585, 396),
    (559, 279, 582, 289)
]

def average_landmarks_three(landmark1, landmark2, landmark3):
    return {
        'x': (landmark1.x + landmark2.x + landmark3.x) / 3,
        'y': (landmark1.y + landmark2.y + landmark3.y) / 3,
        'visibility': (landmark1.visibility + landmark2.visibility + landmark3.visibility) / 3
    }

def average_landmarks(landmark1, landmark2):
    return {
        'x': (landmark1.x + landmark2.x) / 2,
        'y': (landmark1.y + landmark2.y) / 2,
        'visibility': (landmark1.visibility + landmark2.visibility) / 2
    }

def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 25
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[30]  # Custom point
    keypoints[24] = landmarks[32]  # Custom point

    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])
    return coco_keypoints

def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
            cv2.putText(image, str(i//3), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

def draw_coco_skeleton(image, keypoints, pairs):
    for (start, end) in pairs:
        x1, y1, v1 = keypoints[start*3:start*3+3]
        x2, y2, v2 = keypoints[end*3:end*3+3]
        if v1 > 0 and v2 > 0:
            cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

def point_in_rectangle(point, rect):
    x, y = point
    x1, y1, x2, y2 = rect
    return x1 <= x <= x2 and y1 <= y <= y2


# Cargar el video
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
cap = cv2.VideoCapture(video_url)
#cap = cv2.VideoCapture('./caminar2.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('outputFinal.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []
previous_left_ankle = None
previous_right_ankle = None
steps = 0
step_threshold = 25  # Umbral de distancia para considerar un paso (ajusta según sea necesario)
still_threshold = 20  # Umbral de distancia para considerar que la persona está quieta
still_frames = 0
still_frames_threshold = 10  # Número de cuadros consecutivos para considerar que la persona está quieta
is_still = False
movement_direction = None
movement_threshold = 10  # Umbral de movimiento en el eje X para considerar desplazamiento lateral
distance_direction = None
distance_threshold = 6  # Umbral de movimiento en el eje Y para considerar acercamiento/alejamiento


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for rect in rectangles:
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 0), 2)  # Draw rectangles

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        
        draw_coco_keypoints(frame, keypoints)
        skeleton_pairs = [
            (0, 15), (15, 17), (0, 16), (16, 18), # Cabeza 
            (0, 1), #cuello
            (1, 2), (2, 3), (3,4), #Brazo izquierdo
            (1, 5), (6, 7), (5,6), # Brazos derechos
            (1,8), #torso
            (8, 9), (9, 10), (10, 11), (11, 22), (22, 24),  # Pierna izquierda
            (8, 12), (12, 13), (13, 14), (14, 19), (19,21)  # Pierna derecha
        ]
        draw_coco_skeleton(frame, keypoints, skeleton_pairs)

        #left_ankle = np.array([keypoints[35], keypoints[36]])  # Coordenadas del tobillo izquierdo
        #right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho
    
    #con este funciona mejor
        #left_ankle = np.array([keypoints[33], keypoints[34]])  # Coordenadas del tobillo izquierdo
        #right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho

        left_ankle = np.array([keypoints[11*3], keypoints[11*3+1]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[14*3], keypoints[14*3+1]])   # Coordenadas del tobillo derecho

        tip_toe_left = np.array([keypoints[24*3], keypoints[24*3+1]])
        tip_toe_right = np.array([keypoints[21*3], keypoints[21*3+1]])

        heel_left = np.array([keypoints[22*3], keypoints[22*3+1]])
        heel_right = np.array([keypoints[19*3], keypoints[19*3+1]])

        # Promedio de puntos
        average_left_foot = (left_ankle + tip_toe_left + heel_left) / 3
        average_right_foot = (right_ankle + tip_toe_right + heel_right) / 3

        #pintar el punto promedio del pie
        average_left_foot_paint = average_left_foot.astype(int)
        average_right_foot_paint = average_right_foot.astype(int)

        cv2.circle(frame, tuple(average_left_foot_paint), 5, (51, 255, 252), -1)
        cv2.circle(frame, tuple(average_right_foot_paint), 5, (51, 255, 252), -1)

        for rect in rectangles:
            if point_in_rectangle(average_left_foot, rect):
                cv2.putText(frame, 'Left Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            if point_in_rectangle(average_right_foot, rect):
                cv2.putText(frame, 'Right Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if previous_left_ankle is not None and previous_right_ankle is not None:
            left_distance = np.linalg.norm(average_left_foot - previous_left_ankle)
            right_distance = np.linalg.norm(average_right_foot - previous_right_ankle)
            if left_distance > step_threshold or right_distance > step_threshold:
                steps += 1

            if left_distance < still_threshold and right_distance < still_threshold:
                still_frames += 1
            else:
                still_frames = 0

            is_still = still_frames >= still_frames_threshold

            left_movement_x = average_left_foot[0] - previous_left_ankle[0]
            right_movement_x = average_right_foot[0] - previous_right_ankle[0]

            if abs(left_movement_x) > movement_threshold or abs(right_movement_x) > movement_threshold:
                if left_movement_x > 0 and right_movement_x > 0:
                    movement_direction = "Right"
                elif left_movement_x < 0 and right_movement_x < 0:
                    movement_direction = "Left"
                else:
                    movement_direction = "Unknown"
            else:
                movement_direction = "Still"

            left_movement_y = average_left_foot[1] - previous_left_ankle[1]
            right_movement_y = average_right_foot[1] - previous_right_ankle[1]

            if abs(left_movement_y) > distance_threshold or abs(right_movement_y) > distance_threshold:
                if left_movement_y > 0 and right_movement_y > 0:
                    distance_direction = "Closer"
                    steps += 1
                elif left_movement_y < 0 and right_movement_y < 0:
                    distance_direction = "Farther"
                    steps += 1
                else:
                    distance_direction = "Unknown"
            else:
                distance_direction = "Stationary"

        previous_left_ankle = average_left_foot
        previous_right_ankle = average_right_foot

        ann = {
            "image_id": "videoframe",
            "category_id": 1,
            "keypoints": keypoints,
            "score": 1.0
        }
        blazepose_results.append(ann)

    #status_text = f"Steps: {steps} - Still: {'Yes' if is_still else 'No'} - Moving: {movement_direction} - Distance: {distance_direction}"
    status_text = f"Still: {'Yes' if is_still else 'No'} - Moving: {movement_direction} - Distance: {distance_direction}"

    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

with open('blazepose_results_finalDetection.json', 'w') as f:
    json.dump(blazepose_results, f)


Imagen de videoframe pero con las anotaciones de coco

In [6]:
""" # Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen
image = cv2.imread('../yolo/videoframe_0.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Realizar la detección de pose
results = pose.process(image_rgb)
blazepose_results = []

def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints

# Función para dibujar puntos clave en la imagen
def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1) #pinto de color verde los que son coco


# Dibujar las anotaciones de la pose en la imagen
if results.pose_landmarks:
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    #convertir a coco keypoints

    #guardar en annotations
    keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, image.shape)
    #dibujarla
    draw_coco_keypoints(image, keypoints)
    
    ann = {
        "image_id": "videoframe",
        "category_id": 1,  # Persona
        "keypoints": keypoints,
        "score": 1.0  # Puedes ajustar el score según sea necesario
    }
    blazepose_results.append(ann)

#guardar la img
cv2.imwrite('testBlazePose.jpg', image)
# Mostrar la imagen con las poses detectadas
cv2.imshow('BlazePose Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results.json', 'w') as f:
    json.dump(blazepose_results, f) """

In [7]:
""" #con video 
import cv2
import mediapipe as mp
import json

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Función para convertir puntos clave de BlazePose a COCO
def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints

# Función para dibujar puntos clave en la imagen
def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

# Cargar el video
cap = cv2.VideoCapture('./caminata.mp4') #cambiar ruta
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Convertir a COCO keypoints
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        # Dibujar los puntos clave en formato COCO
        draw_coco_keypoints(frame, keypoints)

        ann = {
            "image_id": "videoframe",
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

    # Escribir el cuadro procesado en el video de salida
    out.write(frame)

    # Mostrar el cuadro procesado
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results_video.json', 'w') as f:
    json.dump(blazepose_results, f)
 """

In [23]:
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Función para convertir puntos clave de BlazePose a COCO
def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints

# Función para dibujar puntos clave en la imagen
def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

# Cargar el video
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
cap = cv2.VideoCapture(video_url)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []
previous_left_ankle = None
previous_right_ankle = None
steps = 0
step_threshold = 30  # Umbral de distancia para considerar un paso (ajusta según sea necesario)
still_threshold = 10  # Umbral de distancia para considerar que la persona está quieta
still_frames = 0
still_frames_threshold = 20  # Número de cuadros consecutivos para considerar que la persona está quieta
is_still = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Convertir a COCO keypoints
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        # Dibujar los puntos clave en formato COCO
        draw_coco_keypoints(frame, keypoints)

        # Obtener las posiciones de los tobillos
        left_ankle = np.array([keypoints[45], keypoints[46]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[48], keypoints[49]])  # Coordenadas del tobillo derecho

        # Detectar pasos
        if previous_left_ankle is not None and previous_right_ankle is not None:
            left_distance = np.linalg.norm(left_ankle - previous_left_ankle)
            right_distance = np.linalg.norm(right_ankle - previous_right_ankle)
            if left_distance > step_threshold or right_distance > step_threshold:
                steps += 1

            # Detectar si la persona está quieta
            if left_distance < still_threshold and right_distance < still_threshold:
                still_frames += 1
            else:
                still_frames = 0

            is_still = still_frames >= still_frames_threshold

        previous_left_ankle = left_ankle
        previous_right_ankle = right_ankle

        ann = {
            "image_id": "videoframe",
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

    # Escribir el cuadro procesado en el video de salida
    out.write(frame)

    # Mostrar el cuadro procesado y el contador de pasos y estado de quietud
    status_text = f"Steps: {steps} - Still: {'Yes' if is_still else 'No'}"
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results1.json', 'w') as f:
    json.dump(blazepose_results, f)
 

Codigo que usa 3 puntos en el pie: 

In [22]:
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def average_landmarks_three(landmark1, landmark2, landmark3):
    return {
        'x': (landmark1.x + landmark2.x + landmark3.x) / 3,
        'y': (landmark1.y + landmark2.y + landmark3.y) / 3,
        'visibility': (landmark1.visibility + landmark2.visibility + landmark3.visibility) / 3
    }
def average_landmarks(landmark1, landmark2):
    return {
        'x': (landmark1.x + landmark2.x) / 2,
        'y': (landmark1.y + landmark2.y) / 2,
        'visibility': (landmark1.visibility + landmark2.visibility) / 2
    }
# Función para convertir puntos clave de BlazePose a OpenPose
def convert_blazepose_to_coco(landmarks, image_shape):
    
    keypoints = [None] * 25  # Cambié a 25 para incluir todos los puntos mencionados
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[30]  # Custom point
    keypoints[24] = landmarks[32]  # Custom point

    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    #print(coco_keypoints)
    return coco_keypoints

# Función para dibujar puntos clave en la imagen
def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

# Cargar el video
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
cap = cv2.VideoCapture(video_url)
#cap = cv2.VideoCapture('./caminar2.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []
previous_left_ankle = None
previous_right_ankle = None
steps = 0
step_threshold = 50  # Umbral de distancia para considerar un paso (ajusta según sea necesario)
still_threshold = 40  # Umbral de distancia para considerar que la persona está quieta
still_frames = 0
still_frames_threshold = 15  # Número de cuadros consecutivos para considerar que la persona está quieta
is_still = False
movement_direction = None
movement_threshold = 10  # Umbral de movimiento en el eje X para considerar desplazamiento lateral
distance_direction = None
distance_threshold = 5  # Umbral de movimiento en el eje Y para considerar acercamiento/alejamiento

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Convertir a COCO keypoints
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        # Dibujar los puntos clave en formato COCO
        draw_coco_keypoints(frame, keypoints)

        # Obtener las posiciones de los tobillos
        left_ankle = np.array([keypoints[33], keypoints[34]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho

        # Detectar pasos
        if previous_left_ankle is not None and previous_right_ankle is not None:
            left_distance = np.linalg.norm(left_ankle - previous_left_ankle)
            right_distance = np.linalg.norm(right_ankle - previous_right_ankle)
            if left_distance > step_threshold or right_distance > step_threshold:
                steps += 1

            # Detectar si la persona está quieta
            if left_distance < still_threshold and right_distance < still_threshold:
                still_frames += 1
            else:
                still_frames = 0

            is_still = still_frames >= still_frames_threshold

            # Detectar movimiento lateral
            left_movement_x = left_ankle[0] - previous_left_ankle[0]
            right_movement_x = right_ankle[0] - previous_right_ankle[0]

            if abs(left_movement_x) > movement_threshold or abs(right_movement_x) > movement_threshold:
                if left_movement_x > 0 and right_movement_x > 0:
                    movement_direction = "Right"
                elif left_movement_x < 0 and right_movement_x < 0:
                    movement_direction = "Left"
                else:
                    movement_direction = "Unknown"
            else:
                movement_direction = "Still"

            # Detectar acercamiento/alejamiento
            left_movement_y = left_ankle[1] - previous_left_ankle[1]
            right_movement_y = right_ankle[1] - previous_right_ankle[1]

            if abs(left_movement_y) > distance_threshold or abs(right_movement_y) > distance_threshold:
                if left_movement_y > 0 and right_movement_y > 0:
                    distance_direction = "Closer"  # acerca
                    steps += 1
                elif left_movement_y < 0 and right_movement_y < 0:
                    distance_direction = "Farther"  # aleja
                    steps += 1
                else:
                    distance_direction = "Unknown"
            else:
                distance_direction = "Stationary"  # quiero en eje y

        previous_left_ankle = left_ankle
        previous_right_ankle = right_ankle

        ann = {
            "image_id": "videoframe",
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

    # Agregar la leyenda al fotograma
    status_text = f"Steps: {steps} - Still: {'Yes' if is_still else 'No'} - Moving: {movement_direction} - Distance: {distance_direction}"
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Escribir el cuadro procesado en el video de salida
    out.write(frame)

    # Mostrar el cuadro procesado
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results.json', 'w') as f:
    json.dump(blazepose_results, f)


Codigo de deteccion de pasos con direcciones Izq, Der, Adelante y Retroceso

In [49]:
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Función para convertir puntos clave de BlazePose a OpenPose
""" def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints
 """

def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 25
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[30]  # Custom point
    keypoints[24] = landmarks[32]  # Custom point

    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])
    return coco_keypoints

# Función para dibujar puntos clave en la imagen
def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

# Función para dibujar el esqueleto
def draw_coco_skeleton(image, keypoints, pairs):
    for (start, end) in pairs:
        x1, y1, v1 = keypoints[start*3:start*3+3]
        x2, y2, v2 = keypoints[end*3:end*3+3]
        if v1 > 0 and v2 > 0:
            cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


# Cargar el video
#video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/10_51_54-player10.mp4'

#cap = cv2.VideoCapture(video_url)

#cap = cv2.VideoCapture('./caminar2.mp4')
cap = cv2.VideoCapture('./videoForDetection_v3.mp4')


fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output2.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []
previous_left_ankle = None
previous_right_ankle = None
steps = 0
step_threshold = 70  # Umbral de distancia para considerar un paso (ajusta según sea necesario)
still_threshold = 40  # Umbral de distancia para considerar que la persona está quieta
still_frames = 0
still_frames_threshold = 15  # Número de cuadros consecutivos para considerar que la persona está quieta
is_still = False
movement_direction = None
movement_threshold = 10  # Umbral de movimiento en el eje X para considerar desplazamiento lateral
distance_direction = None
distance_threshold = 5  # Umbral de movimiento en el eje Y para considerar acercamiento/alejamiento

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Convertir a COCO keypoints
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        # Dibujar los puntos clave en formato COCO
        draw_coco_keypoints(frame, keypoints)
        # Dibujar esqueleto
        skeleton_pairs = [
            (0, 15), (15, 17), (0, 16), (16, 18), # Cabeza 
            (0, 1), #cuello
            (1, 2), (2, 3), (3,4), #Brazo izquierdo
            (1, 5), (6, 7), (5,6), # Brazos derechos
            (1,8), #torso
            (8, 9), (9, 10), (10, 11), (11, 22), (22, 24), (11, 24),  # Pierna izquierda
            (8, 12), (12, 13), (13, 14), (14, 19), (19,21), (14, 21)  # Pierna derecha
        ]
        draw_coco_skeleton(frame, keypoints, skeleton_pairs)


        # Obtener las posiciones de los tobillos
        """ left_ankle = np.array([keypoints[33], keypoints[34]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho
 """
        left_ankle = np.array([keypoints[11*3], keypoints[11*3+1]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[14*3], keypoints[14*3+1]])   # Coordenadas del tobillo derecho


        # Detectar pasos
        if previous_left_ankle is not None and previous_right_ankle is not None:
            left_distance = np.linalg.norm(left_ankle - previous_left_ankle)
            right_distance = np.linalg.norm(right_ankle - previous_right_ankle)
            if left_distance > step_threshold or right_distance > step_threshold:
                steps += 1

            # Detectar si la persona está quieta
            if left_distance < still_threshold and right_distance < still_threshold:
                still_frames += 1
            else:
                still_frames = 0

            is_still = still_frames >= still_frames_threshold

            # Detectar movimiento lateral
            left_movement_x = left_ankle[0] - previous_left_ankle[0]
            right_movement_x = right_ankle[0] - previous_right_ankle[0]

            if abs(left_movement_x) > movement_threshold or abs(right_movement_x) > movement_threshold:
                if left_movement_x > 0 and right_movement_x > 0:
                    movement_direction = "Right"
                elif left_movement_x < 0 and right_movement_x < 0:
                    movement_direction = "Left"
                else:
                    movement_direction = "Unknown"
            else:
                movement_direction = "Still"

            # Detectar acercamiento/alejamiento
            left_movement_y = left_ankle[1] - previous_left_ankle[1]
            right_movement_y = right_ankle[1] - previous_right_ankle[1]

            if abs(left_movement_y) > distance_threshold or abs(right_movement_y) > distance_threshold:
                if left_movement_y > 0 and right_movement_y > 0:
                    distance_direction = "Closer"  # acerca
                    steps += 1
                elif left_movement_y < 0 and right_movement_y < 0:
                    distance_direction = "Farther"  # aleja
                    steps += 1
                else:
                    distance_direction = "Unknown"
            else:
                distance_direction = "Stationary"  # quiero en eje y

        previous_left_ankle = left_ankle
        previous_right_ankle = right_ankle

        ann = {
            "image_id": "videoframe",
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

    # Agregar la leyenda al fotograma
    status_text = f"Steps: {steps} - Still: {'Yes' if is_still else 'No'} - Moving: {movement_direction} - Distance: {distance_direction}"
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Escribir el cuadro procesado en el video de salida
    out.write(frame)

    # Mostrar el cuadro procesado
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results2.json', 'w') as f:
    json.dump(blazepose_results, f)


In [55]:
""" #graficar bounding boxes de las marcas del suelo

# Open the video capture
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
cap = cv2.VideoCapture(video_url)
import cv2

# Define area to mark (e.g., a bounding box)
ax1, ay1, ax2, ay2 = 541, 604, 587, 630   # coordenadas de los circulos en el piso
bx1, by1, bx2, by2 = 171, 525, 218, 550 
cx1, cy1, cx2, cy2 = 162, 390, 197, 408
dx1, dy1, dx2, dy2 = 923, 373, 953, 389 
ex1, ey1, ex2, ey2 = 905, 504, 946, 526
fx1, fy1, fx2, fy2 = 768, 299, 793, 318
gx1, gy1, gx2, gy2 = 338, 305, 367, 319
hx1, hy1, hx2, hy2 = 553, 381, 585, 396
ix1, iy1, ix2, iy2 = 559, 279, 582, 289
# Background subtraction (optional)
# ... (implement background subtraction if needed)

while True:
  # Capture frame-by-frame
  ret, frame = cap.read()

  # Check if frame is read correctly
  if not ret:
    print("Error: Unable to capture frame")
    break

  # Apply background subtraction (if implemented)
  # ... (apply background subtraction if used)

  # Draw mark on the frame
  cv2.rectangle(frame, (ax1, ay1), (ax2, ay2), (255, 0, 0), 2)  # Draw a red rectangle
  cv2.rectangle(frame, (bx1, by1), (bx2, by2), (255, 0, 0), 2)
  cv2.rectangle(frame, (cx1, cy1), (cx2, cy2), (255, 0, 0), 2)
  cv2.rectangle(frame, (dx1, dy1), (dx2, dy2), (255, 0, 0), 2)
  cv2.rectangle(frame, (ex1, ey1), (ex2, ey2), (255, 0, 0), 2)
  cv2.rectangle(frame, (fx1, fy1), (fx2, fy2), (255, 0, 0), 2)
  cv2.rectangle(frame, (gx1, gy1), (gx2, gy2), (255, 0, 0), 2)
  cv2.rectangle(frame, (hx1, hy1), (hx2, hy2), (255, 0, 0), 2)
  cv2.rectangle(frame, (ix1, iy1), (ix2, iy2), (255, 0, 0), 2)
  # Display the resulting frame
  cv2.imshow('Frame', frame)

  # Exit if 'q' key is pressed
  if cv2.waitKey(1) == ord('q'):
    break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()

 """

In [39]:
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Define area to mark (e.g., a bounding box)
rectangles = [
    (541, 604, 587, 630),  # Coordenadas de los círculos en el piso
    (171, 525, 218, 550),
    (162, 390, 197, 408),
    (923, 373, 953, 389),
    (905, 504, 946, 526),
    (768, 299, 793, 311),
    (338, 305, 367, 319),
    (553, 381, 585, 396),
    (559, 279, 582, 289)
]

def average_landmarks_three(landmark1, landmark2, landmark3):
    return {
        'x': (landmark1.x + landmark2.x + landmark3.x) / 3,
        'y': (landmark1.y + landmark2.y + landmark3.y) / 3,
        'visibility': (landmark1.visibility + landmark2.visibility + landmark3.visibility) / 3
    }

def average_landmarks(landmark1, landmark2):
    return {
        'x': (landmark1.x + landmark2.x) / 2,
        'y': (landmark1.y + landmark2.y) / 2,
        'visibility': (landmark1.visibility + landmark2.visibility) / 2
    }

def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 25
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[30]  # Custom point
    keypoints[24] = landmarks[32]  # Custom point

    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])
    return coco_keypoints

def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
            cv2.putText(image, str(i//3), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

def draw_coco_skeleton(image, keypoints, pairs):
    for (start, end) in pairs:
        x1, y1, v1 = keypoints[start*3:start*3+3]
        x2, y2, v2 = keypoints[end*3:end*3+3]
        if v1 > 0 and v2 > 0:
            cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

def point_in_rectangle(point, rect):
    x, y = point
    x1, y1, x2, y2 = rect
    return x1 <= x <= x2 and y1 <= y <= y2

# Cargar el video
video_url = 'https://mcp-wildsense.s3.us-east-2.amazonaws.com/videos/7/2024-03-15/11_28_22-player9.mp4'
cap = cv2.VideoCapture(video_url)
#cap = cv2.VideoCapture('./caminar2.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

blazepose_results = []
previous_left_ankle = None
previous_right_ankle = None
steps = 0
step_threshold = 30  # Umbral de distancia para considerar un paso (ajusta según sea necesario)
still_threshold = 40  # Umbral de distancia para considerar que la persona está quieta
still_frames = 0
still_frames_threshold = 15  # Número de cuadros consecutivos para considerar que la persona está quieta
is_still = False
movement_direction = None
movement_threshold = 10  # Umbral de movimiento en el eje X para considerar desplazamiento lateral
distance_direction = None
distance_threshold = 6  # Umbral de movimiento en el eje Y para considerar acercamiento/alejamiento

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for rect in rectangles:
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 0), 2)  # Draw rectangles

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        
        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, frame.shape)
        
        draw_coco_keypoints(frame, keypoints)
        skeleton_pairs = [
            (0, 15), (15, 17), (0, 16), (16, 18), # Cabeza 
            (0, 1), #cuello
            (1, 2), (2, 3), (3,4), #Brazo izquierdo
            (1, 5), (6, 7), (5,6), # Brazos derechos
            (1,8), #torso
            (8, 9), (9, 10), (10, 11), (11, 22), (22, 24),  # Pierna izquierda
            (8, 12), (12, 13), (13, 14), (14, 19), (19,21)  # Pierna derecha
        ]
        draw_coco_skeleton(frame, keypoints, skeleton_pairs)

        left_ankle = np.array([keypoints[35], keypoints[36]])  # Coordenadas del tobillo izquierdo
        right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho

        for rect in rectangles:
            if point_in_rectangle(left_ankle, rect):
                cv2.putText(frame, 'Left Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            if point_in_rectangle(right_ankle, rect):
                cv2.putText(frame, 'Right Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if previous_left_ankle is not None and previous_right_ankle is not None:
            left_distance = np.linalg.norm(left_ankle - previous_left_ankle)
            right_distance = np.linalg.norm(right_ankle - previous_right_ankle)
            if left_distance > step_threshold or right_distance > step_threshold:
                steps += 1

            if left_distance < still_threshold and right_distance < still_threshold:
                still_frames += 1
            else:
                still_frames = 0

            is_still = still_frames >= still_frames_threshold

            left_movement_x = left_ankle[0] - previous_left_ankle[0]
            right_movement_x = right_ankle[0] - previous_right_ankle[0]

            if abs(left_movement_x) > movement_threshold or abs(right_movement_x) > movement_threshold:
                if left_movement_x > 0 and right_movement_x > 0:
                    movement_direction = "Right"
                elif left_movement_x < 0 and right_movement_x < 0:
                    movement_direction = "Left"
                else:
                    movement_direction = "Unknown"
            else:
                movement_direction = "Still"

            left_movement_y = left_ankle[1] - previous_left_ankle[1]
            right_movement_y = right_ankle[1] - previous_right_ankle[1]

            if abs(left_movement_y) > distance_threshold or abs(right_movement_y) > distance_threshold:
                if left_movement_y > 0 and right_movement_y > 0:
                    distance_direction = "Closer"
                    steps += 1
                elif left_movement_y < 0 and right_movement_y < 0:
                    distance_direction = "Farther"
                    steps += 1
                else:
                    distance_direction = "Unknown"
            else:
                distance_direction = "Stationary"

        previous_left_ankle = left_ankle
        previous_right_ankle = right_ankle

        ann = {
            "image_id": "videoframe",
            "category_id": 1,
            "keypoints": keypoints,
            "score": 1.0
        }
        blazepose_results.append(ann)

    status_text = f"Steps: {steps} - Still: {'Yes' if is_still else 'No'} - Moving: {movement_direction} - Distance: {distance_direction}"
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow('BlazePose Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
pose.close()

with open('blazepose_results.json', 'w') as f:
    json.dump(blazepose_results, f)


In [40]:
""" #solo para tener la imagen 
import cv2
import mediapipe as mp
import json
import numpy as np

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Define area to mark (e.g., a bounding box)
rectangles = [
    (541, 604, 587, 630),  # Coordenadas de los círculos en el piso
    (171, 525, 218, 550),
    (162, 390, 197, 408),
    (923, 373, 953, 389),
    (905, 504, 946, 526),
    (768, 299, 793, 311),
    (338, 305, 367, 319),
    (553, 381, 585, 396),
    (559, 279, 582, 289)
]

def average_landmarks_three(landmark1, landmark2, landmark3):
    return {
        'x': (landmark1.x + landmark2.x + landmark3.x) / 3,
        'y': (landmark1.y + landmark2.y + landmark3.y) / 3,
        'visibility': (landmark1.visibility + landmark2.visibility + landmark3.visibility) / 3
    }

def average_landmarks(landmark1, landmark2):
    return {
        'x': (landmark1.x + landmark2.x) / 2,
        'y': (landmark1.y + landmark2.y) / 2,
        'visibility': (landmark1.visibility + landmark2.visibility) / 2
    }

def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 25
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = average_landmarks(landmarks[12], landmarks[11])  # Neck
    keypoints[2] = landmarks[12]  # L shoulder
    keypoints[3] = landmarks[14]  # L elbow 
    keypoints[4] = landmarks[16]  # L hand
    keypoints[5] = landmarks[11]  # R shoulder
    keypoints[6] = landmarks[13]  # R elbow
    keypoints[7] = landmarks[15]  # R hand
    keypoints[8] = average_landmarks(landmarks[24], landmarks[23])  # Hip central pelvis
    keypoints[9] = landmarks[24]  # L hip
    keypoints[10] = landmarks[26]  # L knee
    keypoints[11] = landmarks[28]  # L ankle
    keypoints[12] = landmarks[23]  # R hip
    keypoints[13] = landmarks[25]  # R knee
    keypoints[14] = landmarks[27]  # R ankle
    keypoints[15] = average_landmarks_three(landmarks[5], landmarks[6], landmarks[4])  # Average of points
    keypoints[16] = average_landmarks_three(landmarks[1], landmarks[2], landmarks[3])  # Average of points
    keypoints[17] = landmarks[8]  # Custom point
    keypoints[18] = landmarks[7]  # Custom point
    keypoints[19] = landmarks[29]  # Custom point
    keypoints[21] = landmarks[31]  # Custom point
    keypoints[22] = landmarks[30]  # Custom point
    keypoints[24] = landmarks[32]  # Custom point
    print(len(keypoints))
    print(len(landmarks))
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            if isinstance(point, dict):
                coco_keypoints.extend([point['x'] * image_shape[1], point['y'] * image_shape[0], point['visibility']])
            else:
                coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])
    print(len(coco_keypoints))
    return coco_keypoints

def draw_coco_keypoints(image, keypoints):
    for i in range(0, len(keypoints), 3):
        x, y, v = keypoints[i:i+3]
        if v > 0:
            cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
            cv2.putText(image, str(i//3), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

def draw_coco_skeleton(image, keypoints, pairs):
    for (start, end) in pairs:
        x1, y1, v1 = keypoints[start*3:start*3+3]
        x2, y2, v2 = keypoints[end*3:end*3+3]
        if v1 > 0 and v2 > 0:
            cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

def point_in_rectangle(point, rect):
    x, y = point
    x1, y1, x2, y2 = rect
    return x1 <= x <= x2 and y1 <= y <= y2

# Cargar la imagen
image_path = '../yolo/videoframe_0.jpg'
image = cv2.imread(image_path)

for rect in rectangles:
    cv2.rectangle(image, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 0), 2)  # Draw rectangles

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = pose.process(image_rgb)

if results.pose_landmarks:
    keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, image.shape)
    draw_coco_keypoints(image, keypoints)
    skeleton_pairs = [
        (0, 15), (15, 17), (0, 16), (16, 18), # Cabeza 
        (0, 1), #cuello
        (1, 2), (2, 3), (3,4), #Brazo izquierdo
        (1, 5), (6, 7), (5,6), # Brazos derechos
        (1,8), #torso
        (8, 9), (9, 10), (10, 11), (11, 22), (22, 24),  # Pierna izquierda
        (8, 12), (12, 13), (13, 14), (14, 19), (19,21)  # Pierna derecha
    ]
    draw_coco_skeleton(image, keypoints, skeleton_pairs)

    left_ankle = np.array([keypoints[33], keypoints[34]])  # Coordenadas del tobillo izquierdo
    right_ankle = np.array([keypoints[39], keypoints[40]])  # Coordenadas del tobillo derecho
    
    for rect in rectangles:
        if point_in_rectangle(left_ankle, rect):
            cv2.putText(image, 'Left Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        if point_in_rectangle(right_ankle, rect):
            cv2.putText(image, 'Right Ankle in Area', (rect[0], rect[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow('BlazePose Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
pose.close()
  """

25
33
75
